<p style="font-family: 'Courier New', Courier, monospace; font-size: 40px; font-weight: bold; color: blue;  text-align: center;">
  LoRaWAN Path Loss Measurements in an Indoor Setting: DATA QUERYING from InfluxdB
</p>

# Notebook Summary
This Notebook fetches sensor data from InfluxDB in batches, processes it, and updates a CSV file with combined measurements.

### 1. Libraries and Packages
Imports OS, dotenv, Pandas, InfluxDBClient, time, and pathlib.

In [1]:
# Libraries/Packages Used:
import os                            # For accessing environment variables
from dotenv import load_dotenv       # To load environment variables from the .env file
import pandas as pd                  # For data manipulation and handling timezones
from influxdb import InfluxDBClient  # To interact with InfluxDB (a time-series database)
import time                          # For sleep function between batches
import pathlib                       # For checking if file exists

### 2. Load Environment Variables
Loads .env variables and retrieves InfluxDB host, port, and database.


In [2]:
# Load environment variables from the .env file
load_dotenv()

# Retrieve the variables
host = os.getenv('INFLUXDB_HOST')
port = int(os.getenv('INFLUXDB_PORT'))  # port is an integer
database = os.getenv('INFLUXDB_DATABASE')

### 3. Function: fetch_data
Fetches sensor data between start and end times by converting Europe/Berlin to UTC (and vice versa).

In [3]:
def fetch_data(start_time, end_time):
    """
    Fetch sensor data from InfluxDB between specified start and end times.

    Converts input times from Europe/Berlin to UTC for querying, then back to Europe/Berlin for use.
    Returns data as a pandas DataFrame.
    """
    # Initialize the InfluxDB client
    client = InfluxDBClient(host=host, port=port)
    client.switch_database(database)

    # Convert input times (Europe/Berlin) to UTC for the query
    start_time_utc = pd.to_datetime(start_time).tz_localize('Europe/Berlin').tz_convert('UTC').strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time_utc = pd.to_datetime(end_time).tz_localize('Europe/Berlin').tz_convert('UTC').strftime('%Y-%m-%dT%H:%M:%SZ')

    # Querying with the provided start_time and end_time in UTC
    # '>' for start_time to exclude the last fetched timestamp
    query = f"SELECT * FROM sensor_data WHERE time > '{start_time_utc}' AND time <= '{end_time_utc}'"

    result = client.query(query)
    df = pd.DataFrame(list(result.get_points()))

    if not df.empty:
        # Convert 'time' column to datetime with utc=True
        df['time'] = pd.to_datetime(df['time'], utc=True, format='ISO8601').dt.tz_convert('Europe/Berlin')

    return df

### 4. Function: fetch_data_in_batches
Fetches sensor data in 10-day batches with a 2-minute pause between queries.


In [4]:
def fetch_data_in_batches(start_time, end_time):
    """
    Fetch sensor data in batches of 10 days with a 2-minute  break between each batch.
    """
    # Convert start_time to a datetime object
    start_time = pd.to_datetime(start_time)

    # Check if start_time is timezone-naive; if so, localize it
    if start_time.tzinfo is None:
        start_time = start_time.tz_localize('Europe/Berlin')
    else:
        start_time = start_time.tz_convert('Europe/Berlin')

    # Check if end_time is timezone-naive; if so, localize it
    end_time = pd.to_datetime(end_time)
    if end_time.tzinfo is None:
        end_time = end_time.tz_localize('Europe/Berlin')
    else:
        end_time = end_time.tz_convert('Europe/Berlin')

    # Initialize a list to store DataFrames
    df_list = []

    current_start = start_time
    delta = pd.Timedelta(days=10)

    while current_start < end_time:
        current_end = min(current_start + delta, end_time)

        print(f"Fetching data from {current_start} to {current_end}")

        # Fetch data for the current interval
        df = fetch_data(current_start.strftime('%Y-%m-%d %H:%M:%S'), current_end.strftime('%Y-%m-%d %H:%M:%S'))

        if not df.empty:
            df_list.append(df)

        # Sleep for 2 minutes between queries
        if current_end < end_time:
            print("Sleeping for 2 minutes...")
            time.sleep(120)

        # Move to the next interval
        current_start = current_end

    # Combine all DataFrames
    if df_list:
        batch_combined_df = pd.concat(df_list, ignore_index=True)
    else:
        batch_combined_df = pd.DataFrame()

    return batch_combined_df

###  5. Main Data Fetching & CSV Update
Checks for an existing CSV, resumes from the last timestamp if available, fetches new data, cleans it (drops all-NaN columns), aligns, concatenates, and saves the updated CSV.

In [5]:
# File path for the combined data CSV
csv_file_path = '../all_data_files/unsorted_combined_measurements_data.csv'

# Starting campaign time
initial_start_time = '2024-09-26 13:00:00'  # Berlin time

# Check if the CSV file exists
file_exists = pathlib.Path(csv_file_path).exists()

if file_exists:
    # Read existing data
    combined_df = pd.read_csv(csv_file_path, parse_dates=['time'], low_memory=False)
    if not combined_df.empty:
        # Get the last timestamp
        last_timestamp = combined_df['time'].max()
        # Start from the last timestamp
        start_time = last_timestamp
        print(f"\nResuming data fetching from {start_time}.")
    else:
        combined_df = pd.DataFrame()
        start_time = initial_start_time
        print(f"The existing CSV file is empty. Starting data fetching from {start_time}.")
else:
    combined_df = pd.DataFrame()
    start_time = initial_start_time
    print(f"No existing CSV file found. Starting data fetching from {start_time}.")

# End time is the current time (Berlin)
end_time = pd.Timestamp.now(tz='Europe/Berlin')

# Fetch new data in batches (your custom function)
new_data_df = fetch_data_in_batches(start_time, end_time)

# Combine with existing data if we actually got something
if not new_data_df.empty:
    # -----------------------------------------------------------
    # 1. Identify columns in new_data_df that are entirely NaN
    all_na_cols = new_data_df.columns[new_data_df.isna().all()].tolist()
    
    if all_na_cols:
        print("The following columns are entirely NaN in the newly fetched data and will be dropped:\n")
        # Print each column name as an individual list item
        for col in all_na_cols:
            print(f" - {col}")
    
    # 2. Drop those columns from new_data_df
    new_data_df.drop(columns=all_na_cols, inplace=True)

    # 3. Align columns and concatenate with combined_df
    if not combined_df.empty:
        # Determine the union of columns
        all_cols = sorted(set(combined_df.columns) | set(new_data_df.columns))
        
        # Reindex both DataFrames to ensure all columns align
        combined_df = combined_df.reindex(columns=all_cols)
        new_data_df = new_data_df.reindex(columns=all_cols)
        
        # Concatenate the DataFrames
        combined_df = pd.concat([combined_df, new_data_df], ignore_index=True)
    else:
        combined_df = new_data_df.copy()

    # Save the combined DataFrame
    try:
        combined_df.to_csv(csv_file_path, index=False)
        print(f"\nData fetching completed and saved to '{csv_file_path}'.")
    except Exception as e:
        print(f"\nFailed to save data to CSV: {e}")
else:
    print("No new data fetched.")


Resuming data fetching from 2025-03-29 02:01:54.081417+01:00.
Fetching data from 2025-03-29 02:01:54.081417+01:00 to 2025-03-29 12:05:49.416993+01:00
The following columns are entirely NaN in the newly fetched data and will be dropped:

 - uplink_message_rx_metadata_0_frequency_offset
 - uplink_message_rx_metadata_1_channel_index
 - uplink_message_rx_metadata_1_channel_rssi
 - uplink_message_rx_metadata_1_frequency_offset
 - uplink_message_rx_metadata_1_gateway_ids_eui
 - uplink_message_rx_metadata_1_gateway_ids_gateway_id
 - uplink_message_rx_metadata_1_received_at
 - uplink_message_rx_metadata_1_rssi
 - uplink_message_rx_metadata_1_snr
 - uplink_message_rx_metadata_1_time
 - uplink_message_rx_metadata_1_timestamp
 - uplink_message_rx_metadata_1_uplink_token
 - uplink_message_rx_metadata_2_channel_index
 - uplink_message_rx_metadata_2_channel_rssi
 - uplink_message_rx_metadata_2_frequency_offset
 - uplink_message_rx_metadata_2_gateway_ids_eui
 - uplink_message_rx_metadata_2_gateway_i